In [ ]:
import random
import re
import networkx as nx
import matplotlib.pyplot as plt

########################################################################################

def TARMAC(circuit_netlist, potential_signals, triggers, stimuli, max_vector_number):
    
    design_graph = ConstructDesignGraph(circuit_netlist)
    ComputeLogicExpressions(design_graph, potential_signals, triggers, stimuli)
    
    SG = {
        'V': potential_signals + triggers + stimuli,
        'E': {u: set(potential_signals + triggers + stimuli) - {u} for u in potential_signals + triggers + stimuli}
    }
    
    tests = []
    
    for i in range(1, max_vector_number + 1):
        ti = CliqueSampling(SG)
        tests.append(ti)
    
    return tests


########################################################################################

def ConstructDesignGraph(modules, inputs, outputs, wires):
    G = nx.DiGraph()
    
    # Add modules as nodes
    for module in modules:
        G.add_node(module)
        
    # Connect inputs to modules
    for input_pin in inputs:
        G.add_edge(input_pin, input_pin.split('.')[0])
        
    # Connect modules to outputs
    for output_pin in outputs:
        G.add_edge(output_pin.split('.')[0], output_pin)
        
    # Connect wires between modules
    for wire in wires:
        source_module = wire.split('.')[0]
        dest_module = wire.split('.')[1]
        G.add_edge(source_module, dest_module)
    
    return G

def parse_verilog(file_path):
    modules = []
    inputs = []
    outputs = []
    wires = []
    
    with open(file_path, 'r') as file:
        verilog_code = file.read()
    
    # Extract module names
    module_matches = re.findall(r'module\s+(\w+)', verilog_code)
    modules.extend(module_matches)
    
    # Extract input names
    input_matches = re.findall(r'input\s+(\w+)', verilog_code)
    inputs.extend(input_matches)
    
    # Extract output names
    output_matches = re.findall(r'output\s+(\w+)', verilog_code)
    outputs.extend(output_matches)
    
    # Extract wire names
    wire_matches = re.findall(r'wire\s+(\w+)\s*;', verilog_code)
    wires.extend(wire_matches)
    
    return modules, inputs, outputs, wires

# Example Verilog file: design.v
# module AND2(input A, B, output Y);
# module OR2(input A, B, output Y);
# module NOT1(input A, output Y);
# wire A_to_AND, B_to_AND, A_to_OR, B_to_OR, Y_from_AND, Y_from_OR, Y_from_NOT;

# Provide the path to the Verilog file
file_path = 'Dropbox/Spring 2023/Hardware Security/Final Project/HardwareTrojan Project_EE6914/HardwareTrojan Project_EE6914/sample_design/size_4/c2670_T_0.v'

# Parse the Verilog file
modules, inputs, outputs, wires = parse_verilog(file_path)

# Generate the graph design
design_graph = generate_graph_design(modules, inputs, outputs, wires)

# Draw the graph
pos = nx.spring_layout(design_graph)
nx.draw(design_graph, pos, with_labels=True, node_size=1000, node_color='lightblue')
plt.show()

########################################################################################

def ComputeLogicExpressions(design_graph, potential_signals, triggers, stimuli):
    # Implementation specific to compute logic expressions for P, T, S in the design graph
    # Replace with your code or library calls
    pass

########################################################################################

def CliqueSampling(SG):
    cns = True
    P = list(SG.V)
    
    while P:
        v = random.choice(P)
        P.remove(v)
        
        if satisfiable(cns and (v.le == v.rv)):
            cns = cns and (v.le == v.rv)
            P = intersection(P, SG.E(v))
        elif has_single_constraint(cns):
            u = get_single_constraint_variable(cns)
            SG.E[v] = SG.E[v] - {u}
            SG.E[u] = SG.E[u] - {v}
    
    # Use SMT solver to solve cns and return the test
    test = solve_with_SMT_solver(cns)
    return test

def satisfiable(expression):
    return bool(expression)

def has_single_constraint(cns):
    return len(cns.args) == 1

def intersection(expression1, expression2):
    return expression1 and expression2

# Example usage
circuit_netlist = ...  # Provide the circuit netlist
potential_signals = ...  # Provide the potential trigger signals
triggers = ...  # Provide the trigger signals
stimuli = ...  # Provide the stimuli signals
max_vector_number = ...  # Provide the maximum vector number

tests = TARMAC(circuit_netlist, potential_signals, triggers, stimuli, max_vector_number)
print(tests)
